# MODEL
- KNN
- Decision Tree
- SVM

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
df_raw = pd.read_csv('./log2.csv')
display(df_raw)

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18
...,...,...,...,...,...,...,...,...,...,...,...,...
65527,63691,80,13237,80,allow,314,192,122,6,15,4,2
65528,50964,80,13485,80,allow,4680740,67312,4613428,4675,77,985,3690
65529,54871,445,0,0,drop,70,70,0,1,0,1,0
65530,54870,445,0,0,drop,70,70,0,1,0,1,0


# CLEAN DATA

In [4]:
df_raw.dtypes

Source Port              int64
Destination Port         int64
NAT Source Port          int64
NAT Destination Port     int64
Action                  object
Bytes                    int64
Bytes Sent               int64
Bytes Received           int64
Packets                  int64
Elapsed Time (sec)       int64
pkts_sent                int64
pkts_received            int64
dtype: object

In [5]:
df = df_raw#df_raw.loc[df_raw['Action'] != 'reset-both']
df['Action'] = df['Action'].astype('string')

In [55]:
df.dtypes

Source Port              int64
Destination Port         int64
NAT Source Port          int64
NAT Destination Port     int64
Action                  string
Bytes                    int64
Bytes Sent               int64
Bytes Received           int64
Packets                  int64
Elapsed Time (sec)       int64
pkts_sent                int64
pkts_received            int64
dtype: object

In [57]:
df.describe()

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
count,65532.000000,65532.000000,65532.000000,65532.000000,6.553200e+04,6.553200e+04,6.553200e+04,6.553200e+04,65532.000000,65532.000000,65532.000000
mean,49391.969343,10577.385812,19282.972761,2671.049930,9.712395e+04,2.238580e+04,7.473815e+04,1.028660e+02,65.833577,41.399530,61.466505
std,15255.712537,18466.027039,21970.689669,9739.162278,5.618439e+06,3.828139e+06,2.463208e+06,5.133002e+03,302.461762,3218.871288,2223.332271
min,0.000000,0.000000,0.000000,0.000000,6.000000e+01,6.000000e+01,0.000000e+00,1.000000e+00,0.000000,1.000000,0.000000
25%,49183.000000,80.000000,0.000000,0.000000,6.600000e+01,6.600000e+01,0.000000e+00,1.000000e+00,0.000000,1.000000,0.000000
50%,53776.500000,445.000000,8820.500000,53.000000,1.680000e+02,9.000000e+01,7.900000e+01,2.000000e+00,15.000000,1.000000,1.000000
75%,58638.000000,15000.000000,38366.250000,443.000000,7.522500e+02,2.100000e+02,4.490000e+02,6.000000e+00,30.000000,3.000000,2.000000
max,65534.000000,65535.000000,65535.000000,65535.000000,1.269359e+09,9.484772e+08,3.208818e+08,1.036116e+06,10824.000000,747520.000000,327208.000000


In [58]:
lst_cat = ['Source Port', 'Destination Port', 'NAT Source Port', 'NAT Destination Port']
lst_num =  [x for x in list(df.columns) if x not in lst_cat]
print(lst_num)

['Action', 'Bytes', 'Bytes Sent', 'Bytes Received', 'Packets', 'Elapsed Time (sec)', 'pkts_sent', 'pkts_received']


# EDA

## CHECK MISSING VALUES

In [59]:
df.isna().sum()

Source Port             0
Destination Port        0
NAT Source Port         0
NAT Destination Port    0
Action                  0
Bytes                   0
Bytes Sent              0
Bytes Received          0
Packets                 0
Elapsed Time (sec)      0
pkts_sent               0
pkts_received           0
dtype: int64

## Check number unique in category columns

In [27]:
print('TOTAL RECORD :\t', len(df))
for i in lst_cat:
    print(f'{i} N uqnie :\t{df[i].nunique()}')
    


TOTAL RECORD :	 65532
Source Port N uqnie :	22724
Destination Port N uqnie :	3273
NAT Source Port N uqnie :	29152
NAT Destination Port N uqnie :	2533


In [30]:
df.loc[df['pkts_sent'] != 1]

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18
5,51465,443,39975,443,allow,3961,1595,2366,21,16,12,9
6,60513,47094,45469,47094,allow,320,140,180,6,7,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...
65505,35608,443,62915,443,allow,5776,1880,3896,19,272,11,8
65511,58574,443,3429,443,allow,3447,788,2659,13,135,6,7
65526,51710,43069,65147,43069,allow,70,70,0,2,8,2,0
65527,63691,80,13237,80,allow,314,192,122,6,15,4,2


In [62]:
df.loc[df['Source Port'] == 57222]

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
18,57222,53,51448,53,allow,177,94,83,2,30,1,1
40721,57222,445,0,0,drop,66,66,0,1,0,1,0


## แต่ละ Action มีสัดส่วน Unique Port เท่าไหร่

In [81]:
df

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18
...,...,...,...,...,...,...,...,...,...,...,...,...
65527,63691,80,13237,80,allow,314,192,122,6,15,4,2
65528,50964,80,13485,80,allow,4680740,67312,4613428,4675,77,985,3690
65529,54871,445,0,0,drop,70,70,0,1,0,1,0
65530,54870,445,0,0,drop,70,70,0,1,0,1,0


In [85]:
df.loc[df['Source Port'] == 0]

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
1042,0,0,0,0,deny,98,98,0,1,0,1,0
1043,0,0,0,0,deny,98,98,0,1,0,1,0
1044,0,0,0,0,deny,98,98,0,1,0,1,0
1045,0,0,0,0,allow,244,244,0,2,0,2,0
1046,0,0,0,0,deny,98,98,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
64224,0,0,0,0,allow,480,244,236,4,0,2,2
64624,0,0,0,0,allow,600,306,294,6,0,3,3
64625,0,0,0,0,allow,600,306,294,6,0,3,3
64626,0,0,0,0,allow,600,306,294,6,0,3,3


In [96]:
df.loc[df['Source Port'] == 57222]

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
18,57222,53,51448,53,allow,177,94,83,2,30,1,1
40721,57222,445,0,0,drop,66,66,0,1,0,1,0


In [97]:
df.groupby('Source Port')

In [87]:
df.groupby('Source Port').agg('mean')['Bytes']

C:\Users\kimte\AppData\Local\Temp\ipykernel_10324\552370726.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('Source Port').agg('mean')['Bytes']


Source Port
0           242.601156
5            62.000000
20       312610.000000
22          461.000000
25          110.000000
             ...      
65530       170.333333
65531       189.500000
65532       423.333333
65533       171.500000
65534       414.000000
Name: Bytes, Length: 22724, dtype: float64

In [99]:
df.loc[df['Action'] != 'allow']['Source Port'].nunique()

13728

In [100]:
df.loc[df['Action'] == 'allow']['Source Port'].nunique()

16640

In [ ]:
sns.pairplot

In [ ]:
sns.pairplot(df)

# Model

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [8]:
train, test = train_test_split(df, test_size= 0.2, random_state= 42)
y_train = train.pop('Action')
X_train = train

In [9]:
X_train

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
50640,49437,443,4843,443,9621,2347,7274,24,78,12,12
25967,60939,445,0,0,70,70,0,1,0,1,0
3446,49437,53,49111,53,498,204,294,6,30,3,3
10689,64040,12226,50739,12226,70,70,0,1,4,1,0
52462,29021,23,0,0,60,60,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
62570,53035,443,38723,443,20048,9134,10914,96,92,39,57
38158,57942,445,0,0,66,66,0,1,0,1,0
860,35922,443,6231,443,6997,1773,5224,27,17,15,12
15795,56756,443,23896,443,8007,1987,6020,20,15,11,9


In [10]:
test = X_train['Source Port']

## OneHot Encode 1

In [12]:
test = X_train['Source Port']
test = test.values.reshape(-1,1)

enc = OneHotEncoder(sparse_output= True)

port_enc = enc.fit_transform(test).toarray()
df_enc = pd.DataFrame(port_enc)
df_enc

,0,1,2,3,4,5,6,7,8,9,...,20720,20721,20722,20723,20724,20725,20726,20727,20728,20729
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
X_train

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
50640,49437,443,4843,443,9621,2347,7274,24,78,12,12
25967,60939,445,0,0,70,70,0,1,0,1,0
3446,49437,53,49111,53,498,204,294,6,30,3,3
10689,64040,12226,50739,12226,70,70,0,1,4,1,0
52462,29021,23,0,0,60,60,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
62570,53035,443,38723,443,20048,9134,10914,96,92,39,57
38158,57942,445,0,0,66,66,0,1,0,1,0
860,35922,443,6231,443,6997,1773,5224,27,17,15,12
15795,56756,443,23896,443,8007,1987,6020,20,15,11,9


## OneHot Encode 2

In [22]:
data_encoded = pd.get_dummies(df, columns=['Source Port', 'Destination Port', 'NAT Source Port', 'NAT Destination Port'])
data_encoded

,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Source Port_0,Source Port_5,...,NAT Destination Port_64932,NAT Destination Port_65054,NAT Destination Port_65100,NAT Destination Port_65128,NAT Destination Port_65253,NAT Destination Port_65264,NAT Destination Port_65387,NAT Destination Port_65427,NAT Destination Port_65534,NAT Destination Port_65535
0,allow,177,94,83,2,30,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,allow,4768,1600,3168,19,17,10,9,0,0,...,0,0,0,0,0,0,0,0,0,0
2,allow,238,118,120,2,1199,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,allow,3327,1438,1889,15,17,8,7,0,0,...,0,0,0,0,0,0,0,0,0,0
4,allow,25358,6778,18580,31,16,13,18,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65527,allow,314,192,122,6,15,4,2,0,0,...,0,0,0,0,0,0,0,0,0,0
65528,allow,4680740,67312,4613428,4675,77,985,3690,0,0,...,0,0,0,0,0,0,0,0,0,0
65529,drop,70,70,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
65530,drop,70,70,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
data_encoded2 = data_encoded.iloc[:,:10000]
data_encoded2

,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Source Port_0,Source Port_5,...,Source Port_50848,Source Port_50849,Source Port_50850,Source Port_50851,Source Port_50852,Source Port_50853,Source Port_50854,Source Port_50856,Source Port_50857,Source Port_50858
0,allow,177,94,83,2,30,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,allow,4768,1600,3168,19,17,10,9,0,0,...,0,0,0,0,0,0,0,0,0,0
2,allow,238,118,120,2,1199,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,allow,3327,1438,1889,15,17,8,7,0,0,...,0,0,0,0,0,0,0,0,0,0
4,allow,25358,6778,18580,31,16,13,18,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65527,allow,314,192,122,6,15,4,2,0,0,...,0,0,0,0,0,0,0,0,0,0
65528,allow,4680740,67312,4613428,4675,77,985,3690,0,0,...,0,0,0,0,0,0,0,0,0,0
65529,drop,70,70,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
65530,drop,70,70,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data_encoded2.drop('Action', axis=1), data_encoded2['Action'], test_size=0.2, random_state=42)

In [33]:
from sklearn.preprocessing import StandardScaler

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [34]:
# Train SVM model
svm = SVC()
svm.fit(X_train_scaled, y_train)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# Test SVM model
y_pred_svm = svm.predict(X_test_scaled)

# Evaluate SVM model
print(classification_report(y_test, y_pred_svm))